<a href="https://colab.research.google.com/github/Subhaji24/ajee/blob/main/WEB_CAM_DB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install deepface

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.7/87.7 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 49.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.3/88.3 kB 12.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.2/80.2 kB 10.4 MB/s eta 0:00:00
  Created wheel for fire: filename=fire-0.5.0-py2.py3-none-any.whl size=116934 sha256=6d1d5d5ff2be813a85d3f22e732d60533b85c3fced2d0ab03bc04558b273c58b
  Stored in directory: /root/.cache/pip/wheels/90/d4/f7/9404e5db0116bd4d43e5666eaa3e70ab53723e1e3ea40c9a95
Successfully built fire


In [2]:
from deepface import DeepFace

24-02-21 09:23:19 - Directory /root/.deepface created
24-02-21 09:23:19 - Directory /root/.deepface/weights created


In [12]:
# import necessary libraries
from IPython.display import display, Javascript, Image
from google.colab.output import eval_js
from base64 import b64decode, b64encode
import cv2
import numpy as np
from deepface import DeepFace

# function to convert the JavaScript object into an OpenCV image
def js_to_image(js_reply):
    """
    Params:
    js_reply: JavaScript object containing image from webcam
    Returns:
    img: OpenCV BGR image
    """
    # decode base64 image
    image_bytes = b64decode(js_reply.split(',')[1])
    # convert bytes to numpy array
    jpg_as_np = np.frombuffer(image_bytes, dtype=np.uint8)
    # decode numpy array into OpenCV BGR image
    img = cv2.imdecode(jpg_as_np, flags=1)

    return img

# function to check if face exists in database
def check_face_existence(target_image, database_images_paths):
    for image_path in database_images_paths:
        db_image = cv2.imread(image_path)
        result = DeepFace.verify(target_image, db_image)
        if result["verified"]:
            print(f"Face found in {image_path}")
            return True
    print("Face not found in database")
    return False

def take_photo_and_check(webcam_path='/content/photo.jpg', database_images_paths=None, quality=0.8):
    js = Javascript('''
    async function takePhoto(quality) {
        const div = document.createElement('div');
        const capture = document.createElement('button');
        capture.textContent = 'Capture';
        div.appendChild(capture);

        const video = document.createElement('video');
        video.style.display = 'block';
        const stream = await navigator.mediaDevices.getUserMedia({video: true});

        document.body.appendChild(div);
        div.appendChild(video);
        video.srcObject = stream;
        await video.play();

        // Resize the output to fit the video element.
        google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

        // Wait for Capture to be clicked.
        await new Promise((resolve) => capture.onclick = resolve);

        const canvas = document.createElement('canvas');
        canvas.width = video.videoWidth;
        canvas.height = video.videoHeight;
        canvas.getContext('2d').drawImage(video, 0, 0);
        stream.getVideoTracks()[0].stop();
        div.remove();
        return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
    display(js)

    # Capture photo from webcam
    data = eval_js('takePhoto({})'.format(quality))

    # Convert JavaScript data to OpenCV image
    img = js_to_image(data)

    # Check if the captured image is empty
    if img is None:
        print("Error: Failed to capture image from webcam.")
        return

    # Convert image to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Check if database_images_paths is provided
    if database_images_paths is None:
        print("Error: Database image paths not provided.")
        return

    # Check if face exists in database using DeepFace
    if check_face_existence(img, database_images_paths):
        print('Face exists in the database')
    else:
        print('Face does not exist in the database')

    # Save image
    cv2.imwrite(webcam_path, img)
    return webcam_path

# Paths to database images
database_images_paths = [
    "/content/drive/MyDrive/AI/450A4394.JPG",
    "/content/drive/MyDrive/AI/450A4398.JPG",
    "/content/drive/MyDrive/AI/450A4486.JPG",
    "/content/drive/MyDrive/AI/450A4496.JPG"
]

# Call take_photo_and_check function with specified paths
try:
    webcam_path = take_photo_and_check(webcam_path='/content/photo.jpg', database_images_paths=database_images_paths)
    print('Webcam photo saved to {}'.format(webcam_path))
except Exception as err:
    # Errors will be thrown if there are issues with webcam or image processing.
    print(str(err))


<IPython.core.display.Javascript object>

Face found in /content/drive/MyDrive/AI/450A4394.JPG
Face exists in the database
Webcam photo saved to /content/photo.jpg
